In [1]:
import pandas as pd
import numpy as np
import os

In [41]:
df = pd.read_csv('ner_tag.csv', header=None)
df.columns = ['word', 'tag', 'intent']
df

,word,tag,intent
0,취소,CANCEL,10
1,X,CANCEL,10
2,10분,DT,7
3,10시,DT,7
4,11분,DT,7
...,...,...,...
465,개인잔,TUMBLER,8
466,개인컵,TUMBLER,8
467,개인컵,TUMBLER,8
468,개인텀블러,TUMBLER,8


In [19]:
sent_df = pd.read_csv('D:/github/Chatbot_FG/models/intent/Intent_train_data.csv', header=None)
sent_df.columns = ['sentence', 'intent', 'intent_info']
sent_df

,sentence,intent,intent_info
0,빵종류?,0,메뉴판 요구
1,종류?,0,메뉴판 요구
2,음료종류?,0,메뉴판 요구
3,커피종류?,0,메뉴판 요구
4,식사종류?,0,메뉴판 요구
...,...,...,...
10846,초코쿠키 쉐이크 주문 하지마,10,주문취소
10847,딸기 쉐이크 주문 하지마,10,주문취소
10848,레몬 에이드 주문 하지마,10,주문취소
10849,자몽 에이드 주문 하지마,10,주문취소


In [48]:
sentences1 = sent_df['sentence'].unique()

In [49]:
sentences2 = []
for idx, sent in enumerate(sentences1):
    for n in range(11):
        for idx2, intent in enumerate(df['intent']): 
            if intent == n:
                if sent.find(df['word'][idx2]) == 0:
                    sentences2.append(sent.replace(df['word'][idx2], f'<{df["word"][idx2]}:{df["tag"][idx2]}>'))
sentences2

['<빵종류:MENU>?',
 '<종류:MENU>?',
 '<음료종류:MENU>?',
 '<커피종류:MENU>?',
 '<식사종류:MENU>?',
 '<에이드종류:MENU>?',
 '<사이드메뉴:MENU>는 뭐 있어?',
 '<사이드 메뉴:MENU>는 뭐 있어?',
 '<빵종류:MENU>는 뭐 있어?',
 '<종류:MENU>는 뭐 있어?',
 '<음료종류:MENU>는 뭐 있어?',
 '<커피종류:MENU>는 뭐 있어?',
 '<식사종류:MENU>는 뭐 있어?',
 '<에이드종류:MENU>는 뭐 있어?',
 '<사이드메뉴:MENU>는?',
 '<사이드 메뉴:MENU>는?',
 '<빵종류:MENU>는?',
 '<종류:MENU>는?',
 '<음료종류:MENU>는?',
 '<커피종류:MENU>는?',
 '<식사종류:MENU>는?',
 '<에이드종류:MENU>는?',
 '<사이드메뉴:MENU> 다 돼?',
 '<사이드 메뉴:MENU> 다 돼?',
 '<빵종류:MENU> 다 돼?',
 '<종류:MENU> 다 돼?',
 '<음료종류:MENU> 다 돼?',
 '<커피종류:MENU> 다 돼?',
 '<식사종류:MENU> 다 돼?',
 '<에이드종류:MENU> 다 돼?',
 '<사이드메뉴:MENU> 다 되지?',
 '<사이드 메뉴:MENU> 다 되지?',
 '<빵종류:MENU> 다 되지?',
 '<종류:MENU> 다 되지?',
 '<음료종류:MENU> 다 되지?',
 '<커피종류:MENU> 다 되지?',
 '<식사종류:MENU> 다 되지?',
 '<에이드종류:MENU> 다 되지?',
 '<사이드메뉴:MENU>도 있어?',
 '<사이드 메뉴:MENU>도 있어?',
 '<빵종류:MENU>도 있어?',
 '<종류:MENU>도 있어?',
 '<음료종류:MENU>도 있어?',
 '<커피종류:MENU>도 있어?',
 '<식사종류:MENU>도 있어?',
 '<에이드종류:MENU>도 있어?',
 '<사이드메뉴:MENU>도 있나?',
 '<사이드 메뉴:MENU>도 있나?',
 '<빵종류:MEN

In [51]:
# 전처리 모듈 불러오기
from utils.Preprocess import Preprocess
p = Preprocess(word2index_dic=os.path.join('./train_tools/dict', 'chatbot_dict.bin'),
               userdic=os.path.join('./utils', 'train.tsv'))

sequences = []
for sentence in sent_df['sentence'].unique():
    pos = p.pos(sentence)
    sequences.append(pos)
sequences

[[('빵종류', 'NNG'), ('?', 'SF')],
 [('종류', 'NNG'), ('?', 'SF')],
 [('음료종류', 'NNG'), ('?', 'SF')],
 [('커피종류', 'NNG'), ('?', 'SF')],
 [('식사종류', 'NNG'), ('?', 'SF')],
 [('에이드', 'NNG'), ('종류', 'NNG'), ('?', 'SF')],
 [('사이드', 'NNP'),
  ('메뉴', 'NNP'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  ('?', 'SF')],
 [('사이드', 'NNP'),
  ('메뉴', 'NNP'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  ('?', 'SF')],
 [('빵종류', 'NNG'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  ('?', 'SF')],
 [('종류', 'NNG'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  ('?', 'SF')],
 [('음료종류', 'NNG'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  ('?', 'SF')],
 [('커피종류', 'NNG'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  ('?', 'SF')],
 [('식사종류', 'NNG'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  ('?', 'SF')],
 [('에이드', 'NNG'),
  ('종류', 'NNG'),
  ('는', 'JX'),
  ('뭐', 'NP'),
  ('있', 'VX'),
  ('어', 'EF'),
  

In [62]:
sentences3 = pd.DataFrame(sequences)
sentences3

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"(빵종류, NNG)","(?, SF)",None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"(종류, NNG)","(?, SF)",None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"(음료종류, NNG)","(?, SF)",None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"(커피종류, NNG)","(?, SF)",None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"(식사종류, NNG)","(?, SF)",None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10058,"(초코쿠키 쉐이크, NNG)","(주문, NNP)","(하, VV)","(지, EC)","(말, VX)","(아, EC)",None,None,None,None,None,None,None,None,None,None
10059,"(딸기 쉐이크, NNG)","(주문, NNP)","(하, VV)","(지, EC)","(말, VX)","(아, EC)",None,None,None,None,None,None,None,None,None,None
10060,"(레몬 에이드, NNG)","(주문, NNP)","(하, VV)","(지, EC)","(말, VX)","(아, EC)",None,None,None,None,None,None,None,None,None,None
10061,"(자몽 에이드, NNG)","(주문, NNP)","(하, VV)","(지, EC)","(말, VX)","(아, EC)",None,None,None,None,None,None,None,None,None,None
